##RNN 실습 코드 : https://wikidocs.net/64703
#1. 문자 단위 RNN (Char RNN)
#2. 더 많은 데이터로 학습한 문자 단위 RNN (Char RNN)

##1. 문자 단위 RNN (Char RNN)

In [20]:
# 필요한 패키지 임포트
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

### (Step 1) 훈련 데이터셋 전처리

- 입력 데이터와 레이블 데이터에 대해서 문자 집합(vocabulary)을 만듦
- 문자 집합은 중복을 제거한 문자들의 집합



In [68]:
# 문자 시퀀스 apple을 입력받으면 pple!을 출력하는 RNN 구현 (RNN 동작 이해하기 위함)
input_str = 'apple'
label_str = 'pple!'
char_vocab = sorted(list(set(input_str + label_str)))
vocab_size = len(char_vocab)
print('문자 집합의 크기: {}'.format(vocab_size))

문자 집합의 크기: 5


- 현재 문자 집합엔 총 5개의 문자가 있음.
- !, a, e, l, p

In [69]:
# 하이퍼 파라미터 정의
input_size = vocab_size # 원핫 벡터를 사용할 것이므로 입력의 크기는 문자 집합의 크기
hidden_size = 5
output_size = 5
learning_rate = 0.1

In [70]:
# 문자 집합에 고유한 정수를 부여
char_to_index = dict((c, i) for i, c in enumerate(char_vocab))
print(char_to_index)

{'!': 0, 'a': 1, 'e': 2, 'l': 3, 'p': 4}


In [71]:
# 나중에 예측 결과를 다시 문자 시퀀스로 보기 위해서 반대로 정수로부터 문자를 얻을 수 있는 index_to_char을 만듦
index_to_char = {}
for key, value in char_to_index.items():
  index_to_char[value] = key
print(index_to_char)

{0: '!', 1: 'a', 2: 'e', 3: 'l', 4: 'p'}


In [72]:
# 입력 데이터와 레이블 데이터의 각 문자들을 정수로 매핑
x_data = [char_to_index[c] for c in input_str]
y_data = [char_to_index[c] for c in label_str]
print(x_data)
print(y_data)

[1, 4, 4, 3, 2]
[4, 4, 3, 2, 0]


In [73]:
# 파이토치 nn.RNN()은 3차원 텐서를 입력받으므로 배치 차원 추가
# 텐서 연산인 unsqueeze(0)을 통해서도 해결 가능
x_data = [x_data]
y_data = [y_data]
print(x_data)
print(y_data)

[[1, 4, 4, 3, 2]]
[[4, 4, 3, 2, 0]]


In [74]:
# 입력 시퀀스의 각 문자들을 원핫 벡터롤 변환
x_one_hot = [np.eye(vocab_size)[x] for x in x_data]
print(x_one_hot)

[array([[0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.]])]


In [75]:
# 입력 데이터와 레이블 데이터를 텐서로 바꿔줌
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)
print('훈련 데이터의 크기: {}'.format(X.shape))
print('레이블의 크기: {}'.format(Y.shape))

훈련 데이터의 크기: torch.Size([1, 5, 5])
레이블의 크기: torch.Size([1, 5])


### (Step 2) 모델 구현


In [76]:
# RNN 모델 구현
class Net(torch.nn.Module):
  def __init__(self, input_size, hidden_size, output_size):
    super(Net, self).__init__()
    self.rnn = torch.nn.RNN(input_size, hidden_size, batch_first=True) # RNN 셀 구현
    self.fc = torch.nn.Linear(hidden_size, output_size, bias=True) # 출력층 구현
  def forward(self, x): # 구현한 RNN 셀과 출력층을 연결
    x, _status = self.rnn(x)
    x = self.fc(x)
    return x

In [77]:
# 클래스로 정의한 모델을 net에 저장
net = Net(input_size, hidden_size, output_size)

In [78]:
# 입력된 모델에 입력을 넣어서 출력의 크기를 확인
outputs = net(X)
print(outputs.shape) # 3차원 텐서

torch.Size([1, 5, 5])


- (1,5,5)는 차례대로 (배치 차원, 시점(timesteps), 출력의 크기)를 의미


In [79]:
# 정확도를 측정할 땐 view를 이용하여 배치차원과 시점차원을 하나도 만듦
print(outputs.view(-1, input_size).shape) # 2차원 텐서로 변환

torch.Size([5, 5])


- 차원이 (5,5)가 됨을 확인

In [80]:
# 레이블 데이터의 크기 확인
print(Y.shape)
print(Y.view(-1).shape)

torch.Size([1, 5])
torch.Size([5])


In [81]:
# 옵티마이저와 손실함수를 정의
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), learning_rate)

In [82]:
# 총 100번의 에포크를 학습
for i in range(100):
  optimizer.zero_grad()
  outputs = net(X)
  loss = criterion(outputs.view(-1, input_size), Y.view(-1)) # view를 하는 이유응 Batch 차원 제거를 위해
  loss.backward() # 기울기 계산
  optimizer.step() # 아까 optimizer 선언 시 넣어둔 파라미터 업데이트

  # 아래 세 줄을 모델이 실제 어떻게 예측했는지를 확인하기 위한 코드
  result = outputs.data.numpy().argmax(axis=2) # 최종 예측값인 각 timestep별 5차원 벡터에 대해서 가장 높은 값의 인덱스를 선택
  result_str = ''.join([index_to_char[c] for c in np.squeeze(result)])
  print(i, 'loss: ', loss.item(), 'prediction: ', result,
        'true Y: ', y_data, 'prediction str: ', result_str)


0 loss:  1.683633804321289 prediction:  [[2 3 2 3 3]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  elell
1 loss:  1.4411271810531616 prediction:  [[4 4 4 4 4]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  ppppp
2 loss:  1.3209151029586792 prediction:  [[4 4 4 4 4]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  ppppp
3 loss:  1.2215349674224854 prediction:  [[4 4 4 4 4]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  ppppp
4 loss:  1.093176245689392 prediction:  [[4 4 3 4 4]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pplpp
5 loss:  0.9462994337081909 prediction:  [[4 4 3 2 4]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pplep
6 loss:  0.760388970375061 prediction:  [[4 4 3 2 4]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pplep
7 loss:  0.5758867263793945 prediction:  [[4 4 3 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pple!
8 loss:  0.42612218856811523 prediction:  [[4 4 3 2 0]] true Y:  [[4, 4, 3, 2, 0]] prediction str:  pple!
9 loss:  0.2934550642967224 prediction:  [[4 4 3 2 0]] tr

##2. 더 많은 데이터로 학습한 문자 단위 RNN (Char RNN)

In [83]:
# import necessary modules
import torch
import torch.nn as nn
import torch.optim as optim

### (Step 1) 훈련 데이터셋 전처리하기

In [84]:
# 문장 집합을 생성하고 각 문자에 고유한 정수를 부여
sentence = ("if you want to build a ship, don't drum up people together to "
            "collect wood and don't assign them tasks and work, but rather "
            "teach them to long for the endless immensity of the sea.")
char_set = list(set(sentence)) # 중복을 제거한 문자 집합 생성
char_dic = {c: i for i, c in enumerate(char_set)} # 각 문자에 정수 인코딩
print(char_dic)

{' ': 0, ',': 1, 'u': 2, 'r': 3, 's': 4, 'l': 5, 'i': 6, 'm': 7, 'y': 8, "'": 9, 'k': 10, 'h': 11, 'n': 12, '.': 13, 'f': 14, 'e': 15, 'b': 16, 'w': 17, 'c': 18, 'd': 19, 't': 20, 'g': 21, 'p': 22, 'a': 23, 'o': 24}


In [85]:
# 문자 집합의 크기를 확인
dic_size = len(char_dic)
print('문자 집합의 크기: {}'.format(dic_size))

문자 집합의 크기: 25


In [86]:
# 하이퍼파라미터를 설정
hidden_size = dic_size
sequence_length = 10 # 임의 숫자 지정, 샘플을 10개 단위로 끊어서 샘플을 만들 예정
learning_rate = 0.1

In [87]:
# 한칸 쉬프트된 시퀀스로 데이터 구성
x_data = []
y_data = []

for i in range(0, len(sentence)-sequence_length):
  x_str = sentence[i: i+sequence_length]
  y_str = sentence[i+1: i+sequence_length+1]
  print(i, x_str, '->', y_str)

  x_data.append([char_dic[c] for c in x_str]) # x str to index
  y_data.append([char_dic[c] for c in y_str]) # y str to index


0 if you wan -> f you want
1 f you want ->  you want 
2  you want  -> you want t
3 you want t -> ou want to
4 ou want to -> u want to 
5 u want to  ->  want to b
6  want to b -> want to bu
7 want to bu -> ant to bui
8 ant to bui -> nt to buil
9 nt to buil -> t to build
10 t to build ->  to build 
11  to build  -> to build a
12 to build a -> o build a 
13 o build a  ->  build a s
14  build a s -> build a sh
15 build a sh -> uild a shi
16 uild a shi -> ild a ship
17 ild a ship -> ld a ship,
18 ld a ship, -> d a ship, 
19 d a ship,  ->  a ship, d
20  a ship, d -> a ship, do
21 a ship, do ->  ship, don
22  ship, don -> ship, don'
23 ship, don' -> hip, don't
24 hip, don't -> ip, don't 
25 ip, don't  -> p, don't d
26 p, don't d -> , don't dr
27 , don't dr ->  don't dru
28  don't dru -> don't drum
29 don't drum -> on't drum 
30 on't drum  -> n't drum u
31 n't drum u -> 't drum up
32 't drum up -> t drum up 
33 t drum up  ->  drum up p
34  drum up p -> drum up pe
35 drum up pe -> rum up peo
36

- 총 170개의 샘플이 만들어짐.
- 각 샘플의 각 문자들은 고유한 정수로 인코딩이 된 상태


In [88]:
# 첫번째 샘플의 입력 데이터와 레이블 데이터를 출력
print(x_data[0]) # if you wan 에 해당됨
print(y_data[0]) # f you want 에 해당됨

[6, 14, 0, 8, 24, 2, 0, 17, 23, 12]
[14, 0, 8, 24, 2, 0, 17, 23, 12, 20]


In [89]:
# 입력 시퀀스에 대해 원-핫 인코딩 수행
x_one_hot = [np.eye(dic_size)[x] for x in x_data]
# 입력 데이터와 레이블 데이터를 텐서로 변환
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

In [90]:
print('훈련 데이터의 크기 : {}'.format(X.shape))
print('레이블의 크기 : {}'.format(Y.shape))

훈련 데이터의 크기 : torch.Size([170, 10, 25])
레이블의 크기 : torch.Size([170, 10])


In [91]:
# 첫번째 샘플만 출력
print(X[0])

tensor([[0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,

In [92]:
# 레이블 데이터의 첫번째 샘플도 출력
print(Y[0]) # f you want 에 해당

tensor([14,  0,  8, 24,  2,  0, 17, 23, 12, 20])


### (Step 2) 모델 구현하기
- 은닉층 두 개

In [93]:
class Net(torch.nn.Module):
  def __init__(self, input_dim, hidden_dim, layers): # 현재 hidden_size는 dic_size와 같음.
    super(Net, self).__init__()
    self.rnn = torch.nn.RNN(input_dim, hidden_dim, num_layers=layers, batch_first=True)
    self.fc = torch.nn.Linear(hidden_dim, hidden_dim, bias=True)

  def forward(self, x):
    x, _status = self.rnn(x)
    x = self.fc(x)
    return x


In [94]:
net = Net(dic_size, hidden_size, 2) # 모델 선언 시 인자에 2를 전달하여 은닉층을 두 개 쌓음.

In [95]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), learning_rate)

In [96]:
outputs = net(X)
print(outputs.shape) # 3차원 텐서

torch.Size([170, 10, 25])


In [97]:
print(outputs.view(-1, dic_size).shape) # 2차원 텐서로 변환

torch.Size([1700, 25])


In [98]:
# 레이블의 크기
print(Y.shape)
print(Y.view(-1).shape)

torch.Size([170, 10])
torch.Size([1700])


In [99]:
for i in range(100):
  optimizer.zero_grad()
  outputs = net(X) # (170, 10, 25) 크기를 가진 텐서를 매 에포크마다 모델의 입력으로 사용
  loss = criterion(outputs.view(-1, dic_size), Y.view(-1))
  loss.backward()
  optimizer.step()

  # results의 텐서 크기는 (170, 10)
  results = outputs.argmax(dim=2)
  predict_str = ""
  for j, result in enumerate(results):
    if j == 0: # 처음에는 예측 결과를 전부 가져오지만
      predict_str += ''.join([char_set[t] for t in result])
    else: # 그 다음에는 마지막 글자만 반복 추가
      predict_str += char_set[result[-1]]

  print(predict_str)

'''''rw''''''''''w'''''''r''''''w'''''w''''''''''''''''''''''''wr'''w'''''''''''''w'''''w''''''''''''''''''''''''''w'''''''''''''''''w''''''''''''''''''''''''''w''''w'''''''''''''
               o     o       o     o                          o  o         o   o     o                     o     o                        o    o          o                        
                                                                                                                                                              s                    
s.. t..  ..........................................................................................................................................................................
s.er.. wo  eo eor re ot  h  et tt th .ot oe  oee e  e  eh  o r     rot  ret oeth ro  o ..  eo r th      tr.   o roh t  oer roee     rotrorte  eo re  e     re     oe  h rot h re   
toahmoertotoontoaeoooooooooooooooaootooeoototootooooootoootooooaootootoototoottootooeooooootooeotooo

- 처음에는 이상한 예측을 하지만 마지막 에포크에서는 꽤 정확한 문자을 생성